In [1]:
import numpy as np
import scipy.stats
from sympy.stats import Normal, P
from sympy import *
import pandas as pd
import math 


# Manejo de un fichero con enunciados y soluciones, se usa un dataframe
#df = pd.DataFrame()
#encabezado_df = [r"enunciado_latex",r"solucion"]
# Los datos tienen que ser una lista con esta estructura:
#datos = [[r"Texto_en_latex1",r"Texto_en_latex2"],[r"Solucion_en_latex_1",r"Solucion_en_latex_2"]]
# df = df.append(pd.DataFrame(dict(zip(encabezado_df, datos))))
# list(df.enunciado_latex)
# list(df.solucion)
lista_ejercicios = []

def mean_confidence_interval2(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

def media_desviacion_muestra(data=[]):
    #devuelve de una muestra su media y su desviación típica
    m = np.mean((1.0*np.array(data)))
    d = np.std((1.0*np.array(data)))
    return m, d

def tikz_confianza(confianza,critico) :

  conf = confianza
  crit = critico
  a = round((1-conf)/2,3)
  tikz = r"""
  \begin{tikzpicture}[scale=0.8]
    \pgfmathdeclarefunction{gauss}{2}{\pgfmathparse{1/(#2*sqrt(2*pi))*exp(-((x-#1)^2)/(2*#2^2))}}
    \tikzmath{\conf = """+latex(conf)+r"""; \crit= """+latex(crit)+r"""; \a="""+latex(a)+r""");}
    \begin{axis}[no markers, domain=-5:5, samples=100, axis lines=left, height=5cm, width=12cm, xtick={0,\crit}, ytick=\empty, xticklabels = {$0$, $z_{\frac{\alpha}{2}}=\crit$},enlargelimits=false, clip=false, axis on top]
      \addplot [fill=cyan!20, draw=none, domain=-\crit:\crit] {gauss(0,1)} \closedcycle;
      \addplot [very thick,cyan!50!black] {gauss(0,1)};
    \end{axis}
    \node[] at (5.2,1.5) {$\conf$};	
    \draw[-]   (\crit+6.5,1)node[right]{$\a$}  --  (\crit+5.6,0.1) ;
  \end{tikzpicture} \\
  """
  return tikz

def tikz_intervalo(centro, radio) :
  tikz = r""" \\ 
  \begin{tikzpicture}[scale=0.4]
      \tikzmath{\a = -10; \b = 10; \aa = \a -1; \bb = \b + 1 ; \dist = \b - \a; \med = (\a + \b)/2;}
      \draw[very thick] (\a,0) -- (\b,0);
      \path [draw=black, fill=white] (\b,0) circle (2pt);
      \path [draw=black, fill=white] (\a,0.0) circle (2pt);
      \draw[latex-latex] (\a - 1.5,0) -- (\b + 1.5,0) ;
      \draw[shift={(\a,0)},color=black] (0pt,3pt) -- (0pt,-3pt);
      \draw[shift={(\a,0)},color=black] (0pt,0pt) -- (0pt,-3pt) node[below] {$"""+latex(centro -radio)+r"""$};
      \draw[shift={(\med,0)},color=black] (0pt,3pt) -- (0pt,-3pt);
      \draw[shift={(\med,0)},color=black] (0pt,0pt) -- (0pt,-3pt) node[below] {$"""+latex(centro)+r"""$};
      \draw[shift={(\b,0)},color=black] (0pt,3pt) -- (0pt,-3pt);
      \draw[shift={(\b,0)},color=black] (0pt,0pt) -- (0pt,-3pt) node[below] {$"""+latex(centro + radio)+r"""$};
      \draw[decorate,decoration={brace}, thick](\med,0.2)--(\b,0.2) node[above, midway] {$E="""+latex(radio)+r"""$}; 
  \end{tikzpicture} \\
  """
  return tikz

def media_intervalo(confianza, mu , sigma, N) : 
    #Intervalo de confianza de la media, la confianza en probabilidad no porcentaje

    #E=scipy.stats.norm.ppf((1-(1-confianza/100)/2), mu, sigma/np.sqrt(N))-mu
    enunciado_latex = r"""Intervalo de confianza para la media, si la media muestral es: {}, la desviación típica: {}, tamaño de la muestra: {} y el grado de confianza: {}\%. \\ \\ """.format(mu,sigma,N,confianza*100)
    alpha = 1-confianza 
    solucion_latex = r"$\alpha=1-"+latex(confianza)+r"="+latex(round(1-confianza,2))+r"\to \frac{\alpha}{2}="+latex(round(alpha/2,4))+r"$ \\ \\ "
    #pmax = (1-(1-confianza)/2)
    pmax = confianza+(alpha)/2
    zcritico=round(scipy.stats.norm.ppf(pmax),4)
    solucion_latex += r"Valor crítico: \\ $P(Z>z_{\alpha/2})="+latex(round(alpha/2,4))+r"\to P(Z<z_{\alpha/2})="+latex(pmax)+r" \to z_{\alpha/2} ="+latex(zcritico)+r"$ \\ "
    solucion_latex += tikz_confianza(confianza,zcritico)
    E=round(zcritico*sigma/np.sqrt(N),4)
    solucion_latex += r"Error cometido: \\ $E=z_{\alpha/2}\cdot \frac{\sigma}{\sqrt{n}} \to E="+latex(zcritico)+r"\cdot \frac{"+latex(sigma)+r"}{"+latex(np.sqrt(N))+r"}="+latex(E)+r"$ \\ "
    solucion_latex += r"Por tanto el intervalo de confianza será: \\"
    solucion_latex += r"$\left(\overline{x} - E , \overline{x} + E \right)=\left("+latex(mu)+r" - "+latex(E)+r" , "+latex(mu)+r" + "+latex(E)+r" \right)=\left("+latex(round(mu-E,4))+r", "+latex(round(mu+E,4))+r" \right)$ \\ "
    solucion_latex += tikz_intervalo(mu,E)
    return alpha,pmax,zcritico,E,mu-E,mu+E,scipy.stats.norm.interval(confianza, loc=mu, scale=sigma/np.sqrt(N)),enunciado_latex,solucion_latex

def proporcion_intervalo(confianza, aciertos, N) : 
    #E=scipy.stats.norm.ppf((1-(1-confianza/100)/2), mu, sigma/np.sqrt(N))-mu
    enunciado_latex = r"""Intervalo de confianza para la proporción, si los aciertos son: {}, tamaño de la muestra: {} y el grado de confianza: {}\%. \\ \\ """.format(aciertos,N,confianza*100)
    proporcion_muestral = aciertos/N
    alpha = 1-confianza
    solucion_latex = r"$\alpha=1-"+latex(confianza)+r"="+latex(round(1-confianza,2))+r"\to \frac{\alpha}{2}="+latex(round(alpha/2,4))+r"$ \\ \\ "
    #pmax = (1-(1-confianza)/2)
    pmax = confianza+(alpha)/2
    zcritico=round(scipy.stats.norm.ppf(pmax),4)
    solucion_latex += r"Valor crítico: \\ $P(Z>z_{\alpha/2})="+latex(round(alpha/2,4))+r"\to P(Z<z_{\alpha/2})="+latex(pmax)+r" \to z_{\alpha/2} ="+latex(zcritico)+r"$ \\ "
    solucion_latex += tikz_confianza(confianza,zcritico)

    E=round(zcritico*np.sqrt(proporcion_muestral*(1-proporcion_muestral)/N),4)
    solucion_latex += r"Error cometido: \\ $E=z_{\alpha / 2}\cdot \sqrt{\frac{\widehat{p}\cdot\left(1-\widehat{p} \right)}{n}} \to E="+latex(zcritico)+r"\cdot \sqrt{\frac{"+latex(proporcion_muestral)+r"\cdot"+latex(round(1-proporcion_muestral,4))+r"}{"+latex(N)+r"}}="+latex(E)+r"$ \\ "
    solucion_latex += r"Por tanto el intervalo de confianza será: \\"
    solucion_latex += r"$\left(\widehat{p} - E , \widehat{p} + E \right)=\left("+latex(proporcion_muestral)+r" - "+latex(E)+r" , "+latex(proporcion_muestral)+r" + "+latex(E)+r" \right)=\left("+latex(round(proporcion_muestral-E,4))+r", "+latex(round(proporcion_muestral+E,4))+r" \right)$ \\ "
    solucion_latex += tikz_intervalo(proporcion_muestral,E)

    return proporcion_muestral, alpha,pmax,zcritico,E,proporcion_muestral-E,proporcion_muestral+E,scipy.stats.norm.interval(confianza, loc=aciertos/N, scale=np.sqrt(((aciertos/N)*(1-aciertos/N))/N)),enunciado_latex,solucion_latex


def error_max_media(confianza, sigma, E) :
    enunciado_latex = r"""Tamaño de la muestra para estimar la media con un error máximo: {}, si la desviación típica es: {} y el grado de confianza: {}\%. \\ \\ """.format(E,sigma,confianza*100)
    alpha = 1-confianza
    #pmax = (1-(1-confianza)/2)
    pmax = confianza+(alpha)/2
    zcritico=round(scipy.stats.norm.ppf(pmax),4)
    solucion_latex = r"$\alpha=1-"+latex(confianza)+r"="+latex(round(1-confianza,2))+r"\to \frac{\alpha}{2}="+latex(round(alpha/2,4))+r"$ \\ \\ "
    solucion_latex += r"Valor crítico: \\ $P(Z>z_{\alpha/2})="+latex(round(alpha/2,4))+r"\to P(Z<z_{\alpha/2})="+latex(pmax)+r" \to z_{\alpha/2} ="+latex(zcritico)+r"$ \\ "
    solucion_latex += tikz_confianza(confianza,zcritico)
    n=(zcritico*sigma/E)**2
    solucion_latex += r"Error cometido: \\ $E=z_{\alpha/2}\cdot \frac{\sigma}{\sqrt{n}} \to n =\frac{\sigma^2 \cdot z_{\alpha / 2}^2}{E^2} \to n \geq"+latex(math.ceil(n))+r"$ \\ "

    
    return alpha, pmax, round(zcritico,4), round(n,4), math.ceil(n), enunciado_latex, solucion_latex


def error_max_proporcion(confianza, proporcion, E) :
    enunciado_latex = r"""Tamaño de la muestra para estimar la proporcion con un error máximo: {}, si la proporción muestral es: {} y el grado de confianza: {}\%. \\ \\ """.format(E,proporcion,confianza*100)
    alpha = 1-confianza
    #pmax = (1-(1-confianza)/2)
    pmax = confianza+(alpha)/2
    zcritico=round(scipy.stats.norm.ppf(pmax),4)
    solucion_latex = r"$\alpha=1-"+latex(confianza)+r"="+latex(round(1-confianza,2))+r"\to \frac{\alpha}{2}="+latex(round(alpha/2,4))+r"$ \\ \\ "
    solucion_latex += r"Valor crítico: \\ $P(Z>z_{\alpha/2})="+latex(round(alpha/2,4))+r"\to P(Z<z_{\alpha/2})="+latex(pmax)+r" \to z_{\alpha/2} ="+latex(zcritico)+r"$ \\ "
    solucion_latex += tikz_confianza(confianza,zcritico)
    n=(proporcion*(1-proporcion))/(E/zcritico)**2
    solucion_latex += r"Error cometido: \\ $E=z_{\alpha / 2}\cdot \sqrt{\frac{\widehat{p}\cdot\left(1-\widehat{p} \right)}{n}} \to n =\frac{\widehat{p}\cdot\left(1-\widehat{p} \right)\cdot z_{\alpha / 2}^2}{E^2} \to n \geq"+latex(math.ceil(n))+r"$ \\ "

    
    return alpha, pmax, round(zcritico,4), round(n,4), math.ceil(n), enunciado_latex, solucion_latex

In [2]:
#ejemplo 

mu = 6.32
sigma = 2.3
confianza = 0.95
N = 100

print(media_intervalo(confianza, mu , sigma, N)[7])
print(media_intervalo(confianza, mu , sigma, N)[8])



Intervalo de confianza para la media, si la media muestral es: 6.32, la desviación típica: 2.3, tamaño de la muestra: 100 y el grado de confianza: 95.0\%. \\ \\ 
$\alpha=1-0.95=0.05\to \frac{\alpha}{2}=0.025$ \\ \\ Valor crítico: \\ $P(Z>z_{\alpha/2})=0.025\to P(Z<z_{\alpha/2})=0.975 \to z_{\alpha/2} =1.96$ \\ 
  \begin{tikzpicture}[scale=0.8]
    \pgfmathdeclarefunction{gauss}{2}{\pgfmathparse{1/(#2*sqrt(2*pi))*exp(-((x-#1)^2)/(2*#2^2))}}
    \tikzmath{\conf = 0.95; \crit= 1.96; \a=0.025);}
    \begin{axis}[no markers, domain=-5:5, samples=100, axis lines=left, height=5cm, width=12cm, xtick={0,\crit}, ytick=\empty, xticklabels = {$0$, $z_{\frac{\alpha}{2}}=\crit$},enlargelimits=false, clip=false, axis on top]
      \addplot [fill=cyan!20, draw=none, domain=-\crit:\crit] {gauss(0,1)} \closedcycle;
      \addplot [very thick,cyan!50!black] {gauss(0,1)};
    \end{axis}
    \node[] at (5.2,1.5) {$\conf$};	
    \draw[-]   (\crit+6.5,1)node[right]{$\a$}  --  (\crit+5.6,0.1) ;
  \end{tikzpic

### Ejercicios del cuadernillo

In [2]:
enunciado = r"""p71e1-Una muestra aleatoria de 36 personas, empleadas en una gran industria, da el número medio de días al año
que faltan al trabajo es $\overline{x} = 12$ con $s = 4$.
a) Dar una estimación puntual de $\mu$.
b) Tomando un nivel del 99\% dar el intervalo de confianza de $\mu$.
c) Tomando un nivel del 95\% dar el intervalo de confianza de $\mu$."""
N=36
mu=12
s=4
#sigma=round(sqrt(N/(N-1)*s**2),4)
sigma = s

solucion = r"""a) $\mu\approx """+latex(mu)+r"""$ \\ b) """
solucion += media_intervalo(0.99,mu,sigma,N)[8] + r"\\ c) "
solucion += media_intervalo(0.95,mu,sigma,N)[8]

lista_ejercicios.append([enunciado,solucion])


In [3]:
enunciado = r"""p71e2- Cuál debe ser el tamaño de una muestra si se quiere estimar el gasto medio de una familia en electricidad,
mensualmente, con un error menor de 1.500 pesetas y el 95/% como nivel de confianza, sabiendo que el gasto
medio en 1999 era de 13.300 ptas. con una desviación de 1.200 ptas."""

E=1500
mu=13300
sigma=1200

solucion = error_max_media(0.95,sigma,E)[6]
#solucion = r"""Deberá ser: """+latex(mu)+r"""$ \\ b) """

lista_ejercicios.append([enunciado,solucion])


In [4]:
enunciado = r"""p71e3- Una muestra aleatoria de 100 bombillas producidas por una determinada fábrica, tiene una vida media de
1.280 horas con una desviación típica de 140 horas:
i) Estimar la duración media de las bombillas fabricas por esa fábrica.
ii) Dar el intervalo de confianza, a un nivel del 0,95 de la estimación anterior."""
N=100
mu=1280
sigma=140

solucion = r"""i) $\mu\approx """+latex(mu)+r"""$ \\ ii) """
solucion += media_intervalo(0.95,mu,sigma,N)[8] 

lista_ejercicios.append([enunciado,solucion])


In [5]:
enunciado = r"""p71e4- La dirección de un hotel quiere saber el número de días que, de promedio, sus clientes se hospedan en él .
Toma una muestra de 400 clientes y obtiene una media de 5,4 días con una desviación típica de 2 días.
i) Estimar el número medio de días que los clientes permanecen en el hotel.
ii) Hallar el intervalo de confianza de la estimación anterior a un nivel del 90\%"""
N=400
mu=5.4
sigma=2

solucion = r"""i) $\mu\approx """+latex(mu)+r"""$ \\ ii) """
solucion += media_intervalo(0.9,mu,sigma,N)[8] 

lista_ejercicios.append([enunciado,solucion])


In [6]:
enunciado = r"""p71e5- Una muestra aleatoria de 225 votantes tiene 135 partidarios para el pago de impuestos destinados a la mejora
de la ciudad.
i) Efectuar una estimación puntual del porcentaje de votantes que están a favor de pagar el nuevo
impuesto.
ii) Dar el intervalo de confianza, a un nivel del 0,95 de la estimación anterior."""
N=225
aciertos=135


solucion = r"""i) $p \approx \widehat{p} ="""+latex(round((aciertos/N)*100),4)+r"""$\% \\ ii) """
solucion += proporcion_intervalo(0.95,aciertos,N)[9] 


lista_ejercicios.append([enunciado,solucion])


In [7]:
enunciado = r"""p71e6- El valor de diez cheques al descubierto ha sido de 670, 1400, 350, 1700, 2000, 1700, 530, 710, 1200 y 1000
euros. Si estas diez denuncias se consideran como una muestra del total de todas las que por ese motivo se
realizan a lo largo de un año, ¿cuál será el valor medio de los cheques al descubierto que se expedirán."""

datos = [670, 1400, 350, 1700, 2000, 1700, 530, 710, 1200, 1000]
N=10

solucion = r"""Calculamos la media estadística para estimar la media: $\overline{x}=\frac{{\sum_{i=1}^n x_i }}{n} ="""+latex(media_desviacion_muestra(datos)[0])+r"""$ \\"""

lista_ejercicios.append([enunciado,solucion])

In [8]:
enunciado = r"""p71e7- De 120 alumnos, la proporción de que tengan dos o más hermanos es de 48/120. Indica los parámetros de la
distribución a las que se ajustarían las muestras de tamaño 30."""
str_p='48/120'
p = S(str_p)
n = 30
str_s= "sqrt(("+str_p+"*(1-"+str_p+"))/"+str(n)+")"


solucion = r"""La distribución muestral $\widehat{p} \rightarrow N \left ( p , \sqrt{ \frac{p \cdot (1-p)} {n}}\right )$. Luego: \\ """
solucion += r"""$p \approx \widehat{p}="""+latex(S(str_p, evaluate=False))+r"""="""+latex(S(str_p))+r"""$ \\ """
solucion += r"""$\sigma \approx \sqrt{\frac{"""+latex(S(str_p, evaluate=False))
solucion += r"""\cdot \left(1-"""+latex(S(str_p, evaluate=False))+r"""\right)}{"""+latex(n)+r"""}}="""+latex(S(str_s))+r"""$ \\ """

lista_ejercicios.append([enunciado,solucion])


In [9]:
enunciado = r"""p71e8- En cierto instituto de Enseñanza Secundaria hay matriculados 800 alumnos. A una muestra seleccionada
aleatoriamente de un 15\% de ellos, se les preguntó si utilizaban la cafetería del centro. Contestaron
negativamente un total de 24 alumnos. Estima el porcentaje de alumnado que utiliza la cafetería del instituto.
Determina, con un confianza del 99\%, el error máximo cometido con dicha estimación"""
N=15*800/100
aciertos=15*800/100-24


solucion = r"""i) Tamaño de la muestra:$"""+latex(N)+r"""$. Alumnos de la muestra que utiliza la cafetería: $"""+latex(aciertos)+r"""\to $ \\ """
solucion += r"""$p \approx \widehat{p} ="""+latex(round((aciertos/N)*100),4)+r"""$\% \\ ii) """
solucion += proporcion_intervalo(0.95,aciertos,N)[9]+ r"\\ Por tanto, el error máximo es: "  
solucion += latex(proporcion_intervalo(0.95,aciertos,N)[4]) 

lista_ejercicios.append([enunciado,solucion])


In [10]:
enunciado = r"""p71e9- Para estimar la proporción de las familias de una determinada ciudad que poseen microondas, 
se quiere
utilizar una muestra aleatoria de medida n. Calcula el valor mínimo de n para garantizar que, a un nivel de
confianza del 95\%, el error en la estimación sea menor que 0,005. (Como se desconoce la proporción, se ha de
tomar el caso más desfavorable que será 0,5)"""

E=0.005
proporcion=0.5


solucion = error_max_proporcion(0.95,proporcion,E)[6]
solucion

lista_ejercicios.append([enunciado,solucion])


In [11]:
df=pd.DataFrame(lista_ejercicios,columns=['enunciado','solucion'])
df.to_csv("ejercicios_intervalos.csv")
df

,enunciado,solucion
0,"p71e1-Una muestra aleatoria de 36 personas, em...",a) $\mu\approx 12$ \\ b) $\alpha=1-0.99=0.01\t...
1,p71e2- Cuál debe ser el tamaño de una muestra ...,$\alpha=1-0.95=0.05\to \frac{\alpha}{2}=0.025$...
2,p71e3- Una muestra aleatoria de 100 bombillas ...,i) $\mu\approx 1280$ \\ ii) $\alpha=1-0.95=0.0...
3,p71e4- La dirección de un hotel quiere saber e...,i) $\mu\approx 5.4$ \\ ii) $\alpha=1-0.9=0.1\t...
4,p71e5- Una muestra aleatoria de 225 votantes t...,i) $p \approx \widehat{p} =60$\% \\ ii) $\alph...
5,p71e6- El valor de diez cheques al descubierto...,Calculamos la media estadística para estimar l...
6,"p71e7- De 120 alumnos, la proporción de que te...",La distribución muestral $\widehat{p} \rightar...
7,p71e8- En cierto instituto de Enseñanza Secund...,i) Tamaño de la muestra:$120.0$. Alumnos de la...
8,p71e9- Para estimar la proporción de las famil...,$\alpha=1-0.95=0.05\to \frac{\alpha}{2}=0.025$...


### Septiembre 2012
La cantidad de refresco que se sirve en cada vaso a la entrada de unos cines está
normalmente distribuida con una desviación típica de 15 ml. Hemos medido las cantidades en los vasos de
los 25 asistentes de una determinada sesión que compraron un refresco y hemos obtenido un promedio de
200,8 ml. Fijado un nivel de confianza del 90%, calcular el intervalo de confianza para la media de la
cantidad de refresco que se sirve en cada vaso.

In [3]:
mu = 200.8
sigma = 15
N = 25
confianza = 0.90

lista_ejercicios.append(media_intervalo(confianza, mu , sigma, N)[7:])
scipy.stats.norm.interval(confianza, loc=mu, scale=sigma/np.sqrt(N)), media_intervalo(confianza, mu , sigma, N)



((195.8654391191456, 205.73456088085442),
 (0.09999999999999998,
  0.95,
  1.6449,
  4.9347,
  195.86530000000002,
  205.7347,
  (195.8654391191456, 205.73456088085442),
  'Intervalo de confianza para la media, si la media muestral es: 200.8, la desviación típica: 15, tamaño de la muestra: 25 y el grado de confianza: 90.0\\%. \\\\ \\\\ ',
  '$\\alpha=1-0.9=0.1\\to \\frac{\\alpha}{2}=0.05$ \\\\ \\\\ Valor crítico: \\\\ $P(Z>z_{\\alpha/2})=0.05\\to P(Z<z_{\\alpha/2})=0.95 \\to z_{\\alpha/2} =1.6449$ \\\\ \n  \\begin{tikzpicture}[scale=0.8]\n    \\pgfmathdeclarefunction{gauss}{2}{\\pgfmathparse{1/(#2*sqrt(2*pi))*exp(-((x-#1)^2)/(2*#2^2))}}\n    \\tikzmath{\\conf = 0.9; \\crit= 1.6449; \\a=0.05);}\n    \\begin{axis}[no markers, domain=-5:5, samples=100, axis lines=left, height=5cm, width=12cm, xtick={0,\\crit}, ytick=\\empty, xticklabels = {$0$, $z_{\\frac{\\alpha}{2}}=\\crit$},enlargelimits=false, clip=false, axis on top]\n      \\addplot [fill=cyan!20, draw=none, domain=-\\crit:\\crit] {

### Septiembre 2017

La duración de las bombillas de un fabricante es una variable aleatoria con distribución
normal de desviación típica 75 horas. Decidimos tomar un tamaño de la muestra igual a 150, comprobamos la duración de cada
bombilla y calculamos su promedio, que resulta ser 1053 horas. Calcular el intervalo de confianza al 98\%
para la media de la duración de las bombillas del fabricante.

In [4]:
mu = 1053
sigma = 75
N = 150
confianza = 0.98

lista_ejercicios.append(media_intervalo(confianza, mu , sigma, N)[7:])
scipy.stats.norm.interval(confianza, loc=mu, scale=sigma/np.sqrt(N)), media_intervalo(confianza, mu , sigma, N)


((1038.7540868609788, 1067.2459131390212),
 (0.020000000000000018,
  0.99,
  2.3263,
  14.2456,
  1038.7544,
  1067.2456,
  (1038.7540868609788, 1067.2459131390212),
  'Intervalo de confianza para la media, si la media muestral es: 1053, la desviación típica: 75, tamaño de la muestra: 150 y el grado de confianza: 98.0\\%. \\\\ \\\\ ',
  '$\\alpha=1-0.98=0.02\\to \\frac{\\alpha}{2}=0.01$ \\\\ \\\\ Valor crítico: \\\\ $P(Z>z_{\\alpha/2})=0.01\\to P(Z<z_{\\alpha/2})=0.99 \\to z_{\\alpha/2} =2.3263$ \\\\ \n  \\begin{tikzpicture}[scale=0.8]\n    \\pgfmathdeclarefunction{gauss}{2}{\\pgfmathparse{1/(#2*sqrt(2*pi))*exp(-((x-#1)^2)/(2*#2^2))}}\n    \\tikzmath{\\conf = 0.98; \\crit= 2.3263; \\a=0.01);}\n    \\begin{axis}[no markers, domain=-5:5, samples=100, axis lines=left, height=5cm, width=12cm, xtick={0,\\crit}, ytick=\\empty, xticklabels = {$0$, $z_{\\frac{\\alpha}{2}}=\\crit$},enlargelimits=false, clip=false, axis on top]\n      \\addplot [fill=cyan!20, draw=none, domain=-\\crit:\\crit] {g

In [5]:
lista_ejercicios

[('Intervalo de confianza para la media, si la media muestral es: 200.8, la desviación típica: 15, tamaño de la muestra: 25 y el grado de confianza: 90.0\\%. \\\\ \\\\ ',
  '$\\alpha=1-0.9=0.1\\to \\frac{\\alpha}{2}=0.05$ \\\\ \\\\ Valor crítico: \\\\ $P(Z>z_{\\alpha/2})=0.05\\to P(Z<z_{\\alpha/2})=0.95 \\to z_{\\alpha/2} =1.6449$ \\\\ \n  \\begin{tikzpicture}[scale=0.8]\n    \\pgfmathdeclarefunction{gauss}{2}{\\pgfmathparse{1/(#2*sqrt(2*pi))*exp(-((x-#1)^2)/(2*#2^2))}}\n    \\tikzmath{\\conf = 0.9; \\crit= 1.6449; \\a=0.05);}\n    \\begin{axis}[no markers, domain=-5:5, samples=100, axis lines=left, height=5cm, width=12cm, xtick={0,\\crit}, ytick=\\empty, xticklabels = {$0$, $z_{\\frac{\\alpha}{2}}=\\crit$},enlargelimits=false, clip=false, axis on top]\n      \\addplot [fill=cyan!20, draw=none, domain=-\\crit:\\crit] {gauss(0,1)} \\closedcycle;\n      \\addplot [very thick,cyan!50!black] {gauss(0,1)};\n    \\end{axis}\n    \\node[] at (5.2,1.5) {$\\conf$};\t\n    \\draw[-]   (\\crit+6.

### Cálculo del tamaño de una muestra

Se desea realizar una investigación para estimar el peso medio de los recién nacidos de madres fumadoras. Se admite un error máximo de 50 gramos, con una confianza del 95%. Si por estudios anteriores se sabe que la desviación típica del peso medio de tales recién nacidos es de 400 gramos, ¿qué tamaño mínimo de muestra se necesita en la investigación?


In [11]:
import math 

E = 50
confianza = 0.91
sigma = 400

def error_max_media(confianza, sigma, E) :
  alpha = 1-confianza
  #pmax = (1-(1-confianza)/2)
  pmax = confianza+(alpha)/2
  zcritico=scipy.stats.norm.ppf(pmax)
  n=(zcritico*sigma/E)**2
  return alpha, pmax, round(zcritico,4), round(n,4), math.ceil(n)

error_max_media(confianza, sigma, E)


def error_max_proporcion(confianza, proporcion, E) :
  alpha = 1-confianza
  #pmax = (1-(1-confianza)/2)
  pmax = confianza+(alpha)/2
  zcritico=scipy.stats.norm.ppf(pmax)
  n=(proporcion*(1-proporcion))/(E/zcritico)**2
  return alpha, pmax, round(zcritico,4), round(n,4), math.ceil(n)

error_max_media(confianza, sigma, E)

(0.08999999999999997, 0.9550000000000001, 1.6954, 183.9599, 184)

In [10]:
E = 0.04
proporcion = 0.5
confianza = 0.91

error_max_proporcion(confianza, proporcion, E)

NameError: name 'math' is not defined

- Deseamos valorar el grado de conocimientos en historia de una población
de varios miles de estudiantes. Sabemos
que σ = 2,3. Nos proponemos estimar μ pasando una prueba a 100
estudiantes.
a) Calcular el intervalo característico
para $\overline{x}$
correspondiente a una probabilidad de 0,95

- Una vez realizada la prueba a 100 estudiantes concretos, se ha obtenido una
media $\overline{x}$ = 6,32.
b) Hallar el intervalo de confianza de
μ con un nivel de confianza del
95 %.


### Junio 2018 Estimación de la proporción por intervalo de confianza

Para estimar la proporción de personas con sobrepeso en una población se ha tomado una
muestra aleatoria simple de tamaño 100 personas, de las cuales 21 tienen sobrepeso. Calcular el intervalo
de confianza al 96% para la proporción de personas con sobrepeso en la población.



In [40]:
proporcion_intervalo(0.96, 21, 100)

(0.21,
 0.040000000000000036,
 0.98,
 2.0537,
 0.0836,
 0.1264,
 0.29359999999999997,
 (0.12634911517884823, 0.29365088482115176),
 'Intervalo de confianza para la proporción, si los aciertos son: 21, tamaño de la muestra: 100 y el grado de confianza: 96.0\\%. \\\\ \\\\ ',
 '$\\alpha=1-0.96=0.04\\to \\frac{\\alpha}{2}=0.02$ \\\\ \\\\ Valor crítico: \\\\ $P(Z>z_{\\alpha/2})=0.02\\to P(Z<z_{\\alpha/2})=0.98 \\to z_{\\alpha/2} =2.0537$ \\\\ \n  \\begin{tikzpicture}[scale=0.8]\n    \\pgfmathdeclarefunction{gauss}{2}{\\pgfmathparse{1/(#2*sqrt(2*pi))*exp(-((x-#1)^2)/(2*#2^2))}}\n    \\tikzmath{\\conf = 0.96; \\crit= 2.0537; \\a=0.02);}\n    \\begin{axis}[no markers, domain=-5:5, samples=100, axis lines=left, height=5cm, width=12cm, xtick={0,\\crit}, ytick=\\empty, xticklabels = {$0$, $z_{\\frac{\\alpha}{2}}=\\crit$},enlargelimits=false, clip=false, axis on top]\n      \\addplot [fill=cyan!20, draw=none, domain=-\\crit:\\crit] {gauss(0,1)} \\closedcycle;\n      \\addplot [very thick,cyan!50!b

## Ejercicios de distribuciones muestrales


Se ha seleccionado una muestra al azar de 50 mujeres de una población de mayores
de 18 años. Se desconoce la talla media de la población, pero en la muestra se ha
observado que la media de las 50 tallas es 1,60 m. Si se sabe por otros estudios que
la desviación típica en la población es de 3,3 cm, determina la probabilidad de que la
media de la población no difiere en más de 1 cm de la de la muestra.


In [10]:
n = 50

media_muestral = 1.60

sigma = 3.3

error_maximo = 1

Z = Normal('Z',0,1)
X = Normal('X',media_muestral,sigma/sqrt(n))

P(And(X>media_muestral-error_maximo,X<media_muestral+error_maximo)), P(And(Z>-1/(sigma/sqrt(n)),Z<1/(sigma/sqrt(n)))), sigma/sqrt(n).evalf(2)
#P(And(3<=X, X<4.5))






(0.967866646858422, 0.967866646858422, 0.466479492187500)

In [19]:
error_max_proporcion(0.95,0.20,0.02)

(0.050000000000000044,
 0.975,
 1.96,
 1536.64,
 1537,
 'Tamaño de la muestra para estimar la proporcion con un error máximo: 0.02, si la proporción muestral es: 0.2 y el grado de confianza: 95.0\\%. \\\\ \\\\ ',
 '$\\alpha=1-0.95=0.05\\to \\frac{\\alpha}{2}=0.025$ \\\\ \\\\ Valor crítico: \\\\ $P(Z>z_{\\alpha/2})=0.025\\to P(Z<z_{\\alpha/2})=0.975 \\to z_{\\alpha/2} =1.96$ \\\\ \n  \\begin{tikzpicture}[scale=0.8]\n    \\pgfmathdeclarefunction{gauss}{2}{\\pgfmathparse{1/(#2*sqrt(2*pi))*exp(-((x-#1)^2)/(2*#2^2))}}\n    \\tikzmath{\\conf = 0.95; \\crit= 1.96; \\a=0.025);}\n    \\begin{axis}[no markers, domain=-5:5, samples=100, axis lines=left, height=5cm, width=12cm, xtick={0,\\crit}, ytick=\\empty, xticklabels = {$0$, $z_{\\frac{\\alpha}{2}}=\\crit$},enlargelimits=false, clip=false, axis on top]\n      \\addplot [fill=cyan!20, draw=none, domain=-\\crit:\\crit] {gauss(0,1)} \\closedcycle;\n      \\addplot [very thick,cyan!50!black] {gauss(0,1)};\n    \\end{axis}\n    \\node[] at (5.2,1.